In [2]:
import requests
import os
from collections import Counter, defaultdict
from itertools import chain, combinations

In [2]:
def download_document(file_name, document_url):

   if os.path.exists(file_name):

       pass

   else:

       response = requests.get(document_url)

       if response.status_code == 200:

           with open(file_name, 'wb') as f:

               f.write(response.content)

       else:

           print(f'Failed to download the document. Status code: {response.status_code}')

In [3]:
file_name = 'orders.txt'

document_url = 'https://drive.google.com/uc?id=1IOPTVq2ooQfZRkF3rAjGkTjRtbotG7FF'

download_document(file_name, document_url)

In [8]:
def read_orders(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        orders = file.read().split('\n\n')
        orders = [order.split('@@@') for order in orders]
    return orders

def count_products(orders):
    product_count = Counter(chain.from_iterable(orders))
    return product_count

def calculate_support(orders, product1, product2):
    support_count = sum(1 for order in orders if product1 in order and product2 in order)
    support = support_count / len(orders)
    return support

def generate_association_rules(orders, min_confidence, min_support):
    product_count = count_products(orders)

    for order in orders:
        for product1, product2 in combinations(order, 2):
            support = calculate_support(orders, product1, product2)
            if support >= min_support:
                confidence1 = support / product_count[product1]
                confidence2 = support / product_count[product2]

                if confidence1 >= min_confidence:
                    print(f"{product1} => {product2} ({confidence1:.2f}% confidence), {int(support*len(orders))} support")
                if confidence2 >= min_confidence:
                    print(f"{product2} => {product1} ({confidence2:.2f}% confidence), {int(support*len(orders))} support")

In [5]:
orders = read_orders('orders.txt')
min_confidence = 0.45
min_support = 0.15

In [ ]:
generate_association_rules(orders, min_confidence, min_support)

In [5]:
import requests
import os
from collections import defaultdict, Counter
from itertools import combinations

def download_document(file_name, document_url):
    if os.path.exists(file_name):
        print('File already downloaded.')
    else:
        response = requests.get(document_url)
        if response.status_code == 200:
            with open(file_name, 'wb') as f:
                f.write(response.content)
            print('File downloaded successfully.')
        else:
            print(f'Failed to download the document. Status code: {response.status_code}')

def read_orders(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        orders = file.read().split('\n\n')
        orders = [order.split('@@@') for order in orders]
    return orders

def count_products(orders):
    return Counter(chain.from_iterable(orders))

def get_frequent_itemsets(orders, min_support):
    total_orders = len(orders)
    itemsets = defaultdict(int)
    
    # Count individual items
    for order in orders:
        for item in set(order):
            itemsets[frozenset([item])] += 1
    
    # Filter out non-frequent itemsets
    itemsets = {itemset: count for itemset, count in itemsets.items() if count / total_orders >= min_support}
    
    k = 2
    while True:
        new_combinations = defaultdict(int)
        
        for order in orders:
            for combination in combinations(set(order), k):
                comb_set = frozenset(combination)
                if all(frozenset(subset) in itemsets for subset in combinations(comb_set, k-1)):
                    new_combinations[comb_set] += 1
        
        new_combinations = {itemset: count for itemset, count in new_combinations.items() if count / total_orders >= min_support}
        
        if not new_combinations:
            break
        
        itemsets.update(new_combinations)
        k += 1
    
    return itemsets

def generate_association_rules(orders, min_confidence, min_support):
    itemsets = get_frequent_itemsets(orders, min_support)
    total_orders = len(orders)
    
    rules = []
    
    for itemset in itemsets:
        if len(itemset) > 1:
            for antecedent in combinations(itemset, len(itemset) - 1):
                antecedent = frozenset(antecedent)
                consequent = itemset - antecedent
                support = itemsets[itemset] / total_orders
                confidence = itemsets[itemset] / itemsets[antecedent]
                
                if confidence >= min_confidence:
                    rules.append((antecedent, consequent, confidence, support))
    
    for rule in rules:
        antecedent, consequent, confidence, support = rule
        print(f"{set(antecedent)} => {set(consequent)} ({confidence:.2f}% confidence), {int(support * total_orders)} support")

file_name = 'orders.txt'
document_url = 'https://drive.google.com/uc?id=1IOPTVq2ooQfZRkF3rAjGkTjRtbotG7FF'

download_document(file_name, document_url)

# Зчитування та обробка даних
orders = read_orders(file_name)
min_confidence = 0.45
min_support = 0.15


File downloaded successfully.


In [6]:
# Генерація асоціативних правил
generate_association_rules(orders, min_confidence, min_support)

In [12]:
import requests
import os
from collections import defaultdict, Counter
from itertools import combinations, chain

def download_document(file_name, document_url):
    if os.path.exists(file_name):
        print('File already downloaded.')
    else:
        response = requests.get(document_url)
        if response.status_code == 200:
            with open(file_name, 'wb') as f:
                f.write(response.content)
            print('File downloaded successfully.')
        else:
            print(f'Failed to download the document. Status code: {response.status_code}')

def read_orders(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        orders = file.read().strip().split('\n\n')
        orders = [order.split('@@@') for order in orders]
    return orders

def count_products(orders):
    return Counter(chain.from_iterable(orders))

def get_frequent_itemsets(orders, min_support):
    total_orders = len(orders)
    itemsets = defaultdict(int)
    
    for order in orders:
        for item in set(order):
            itemsets[frozenset([item])] += 1
    
    itemsets = {itemset: count for itemset, count in itemsets.items() if count / total_orders >= min_support}
    
    print(f"Initial frequent itemsets: {itemsets}")

    k = 2
    while True:
        new_combinations = defaultdict(int)
        
        for order in orders:
            for combination in combinations(set(order), k):
                comb_set = frozenset(combination)
                if all(frozenset(subset) in itemsets for subset in combinations(comb_set, k-1)):
                    new_combinations[comb_set] += 1
        
        new_combinations = {itemset: count for itemset, count in new_combinations.items() if count / total_orders >= min_support}
        
        if not new_combinations:
            break
        
        itemsets.update(new_combinations)
        k += 1
    
    print(f"Frequent itemsets: {itemsets}")
    return itemsets

def generate_association_rules(orders, min_confidence, min_support):
    itemsets = get_frequent_itemsets(orders, min_support)
    total_orders = len(orders)
    
    rules = []
    
    for itemset in itemsets:
        if len(itemset) > 1:
            for antecedent in combinations(itemset, len(itemset) - 1):
                antecedent = frozenset(antecedent)
                consequent = itemset - antecedent
                support = itemsets[itemset] / total_orders
                confidence = itemsets[itemset] / itemsets[antecedent]
                
                if confidence >= min_confidence:
                    rules.append((antecedent, consequent, confidence, support))
    
    if not rules:
        print("No association rules found.")
    else:
        for rule in rules:
            antecedent, consequent, confidence, support = rule
            print(f"{set(antecedent)} => {set(consequent)} ({confidence*100:.2f}% confidence), {int(support * total_orders)} support")

file_name = 'orders.txt'
document_url = 'https://drive.google.com/uc?id=1IOPTVq2ooQfZRkF3rAjGkTjRtbotG7FF'

download_document(file_name, document_url)

orders = read_orders(file_name)
min_confidence = 0.45
min_support = 0.15

generate_association_rules(orders, min_confidence, min_support)


File already downloaded.
Initial frequent itemsets: {}
Frequent itemsets: {}
No association rules found.
